In [17]:
import math
import nbimporter
import PsychrometricProperty as psy

# Homework #3

### Problem 5.19
A 4000 ft3 room is maintained at 70 F, 50% relative humidity, and 14.7 psia. Mechanical ventilation is provided by a central system, while heating and humidification are performed locally at the zone. The ventilation air is provided at 40 F and 30% relative humidity with a flow rate of 50 cfm. There is a total heat loss from the space through the walls of 5000 Btu/hr. A humidifier adds moisture in the form of saturated vapor at 212 F. Assuming the room air is fully mixed, determine the following: 
a. the amount of moisture addition necessary.\
b. the amount of heating required\
c. The mass of water vapor within the room.\
d. the temperature of surfaces within the room at which moisture will condense. 
e. a plot of the process on psychrometric coordinates 
f. Draw some conclusions from your results. 


#### Solutions:

List all the given information:

In [42]:

P_atm = 14.7 # psia
V_z = 4000 # ft^3
T_z = 70 # space zone teperature, F
RH_z = 50 # spce zone humidity, %

T_SA = 40 # supply air temperature, F
RH_SA = 30 # supply air humidity, %
V_dot_SA = 50 # supply air volume flow rate

q_loss = 5000 # Btu/hr
T_s = 212 # saturated vapor temperature,F
h_s = 1150 # saturated vapor enthalpy,btu/hr

Determine the state for space zone and supply air

In [43]:
h_z = psy.h_ptr(P_atm,T_z,RH_z) # enthalpy of space air, btu/hr
w_z = psy.W_ptr(P_atm,T_z,RH_z) # humidity ratio of space air, lbm_w/lbm_a
v_z = psy.v_ptr(P_atm,T_z,RH_z) # specifiv volume of space air, ft^3/lb

h_SA = psy.h_ptr(P_atm,T_SA,RH_SA) # enthalpy of supply air, btu/hr
w_SA = psy.W_ptr(P_atm,T_SA,RH_SA) # humidity ratio of supply air, lbm_w/lbm_a
v_SA = psy.v_ptr(P_atm,T_SA,RH_SA) # specifiv volume of supply air, ft^3/lb

Calculate the mass of air in the space and mass flow rate of supply air

In [49]:
M_z = V_z /v_z # mass of air in space, lb/hr
m_dot_SA = V_dot_SA * 60/v_SA # mass flow rate of supply air, lb/hr

Based on the conservation of mass of mositure:
$$\dot{m}_{SA} w_{SA} + \dot{m}_s = \dot{m}_{SA} w_{z}$$
The amount of moisture is:
$$\dot{m}_s = \dot{m}_{SA} (w_{z}-w_{SA})$$
a) the amount of moisture required:

In [45]:
m_dot_s = m_dot_SA * (w_z-w_SA)
print('a) The amount of moisture is:', m_dot_s, 'lbm/hr')

a) The amount of moisture is: 1.4812584714955404 lbm/hr


Based on the conservation of energy :
$$\dot{m}_{SA} h_{SA} + \dot{m}_s h_s + \dot{q}_{heat} = \dot{m}_{SA} h_{z} + \dot{q}_{loss}$$
b) the amount of heating required is:
$$\dot{q}_{heat} = \dot{m}_{SA} h_{z} + \dot{q}_{loss}-\dot{m}_{SA} h_{SA} - \dot{m}_s h_s $$

In [46]:
q_dot_heat = m_dot_SA*h_z + q_loss - m_dot_SA*h_SA - m_dot_s*h_s
print('b) The amount of heating required is', q_dot_heat,'Btu/hr')

b) The amount of heating required is 6630.373251292586 Btu/hr


c) The mass of water vapor within the room.

In [51]:
M_w = M_z * w_z
print('c) The mass of water vapor within the room is', M_w, 'lbm')

c) The mass of water vapor within the room is 2.303159746192362 lbm


d) the temperature of surfaces within the room at which moisture will condense is the dewpoint temperature at the zone space temperature and humidity.

In [52]:
T_dp_z = psy.Tdp_ptr(P_atm,T_z,RH_z) #F
print('d)the temperature of surfaces within the room at which moisture will condense is', T_dp_z,'F')

d)the temperature of surfaces within the room at which moisture will condense is 50.59130226051858 F


### Problem 3